In [6]:
from pathlib import Path

import pandas as pd


In [7]:
THIS_FOLDER = Path().resolve()
DATA_FOLDER = THIS_FOLDER / "data"

In [8]:
df = pd.read_parquet(DATA_FOLDER / "metadata.parquet")

In [14]:
dupes = df[df["image_filename"].duplicated(keep=False)].sort_values("image_filename")
dupes

,image_filename,latex,split,image_type,typst
3319167,10220a1920.png,"{ \cal H } _ { \Delta , l } = \frac { 1 } { x ...",test,printed,"cal( H ) _ ( Delta , l ) = ( 1 ) / ( x _ ( 12 ..."
2664061,10220a1920.png,"{ \cal H } _ { \Delta , l } = \frac { 1 } { x ...",train,printed,"cal( H ) _ ( Delta , l ) = ( 1 ) / ( x _ ( 12 ..."
156620,105_edwin.bmp,\frac { b ^ { 2 } c ^ { 2 } - 4 b ^ { 3 } d - ...,train,handwritten,( b ^ ( 2 ) c ^ ( 2 ) - 4 b ^ ( 3 ) d - 4 a c ...
3324611,105_edwin.bmp,\frac { b ^ { 2 } c ^ { 2 } - 4 b ^ { 3 } d - ...,validation,handwritten,( b ^ ( 2 ) c ^ ( 2 ) - 4 b ^ ( 3 ) d - 4 a c ...
3324429,106_edwin.bmp,y ^ { 4 } + y + 1 = 0,validation,handwritten,y ^ ( 4 ) + y + 1 = 0
...,...,...,...,...,...
3324715,formulaire037-equation034.bmp,\beta = \frac { u } { c },validation,handwritten,beta = ( u ) / ( c )
78770,formulaire037-equation054.bmp,q ( t ),train,handwritten,q ( t )
3324511,formulaire037-equation054.bmp,q ( t ),validation,handwritten,q ( t )
49902,formulaire039-equation045.bmp,1 3 6 - 9 1 + 1 4 7 \geq - 1 0 2,train,handwritten,136 - 91 + 147 >= - 102


In [15]:
# Get all duplicate groups
dupes = df[df["image_filename"].duplicated(keep=False)].sort_values("image_filename")

# Check if duplicates have identical data in other columns
for filename in dupes["image_filename"].unique():
    group = df[df["image_filename"] == filename]
    
    # Check if all rows in this group are identical (excluding image_filename)
    other_cols = [col for col in df.columns if col != "image_filename"]
    if not group[other_cols].nunique().eq(1).all():
        print(f"WARNING: Duplicate filename '{filename}' has different data!")
        print(group)
        print()

# Deduplicate - keep first occurrence
df_deduped = df.drop_duplicates(subset="image_filename", keep="first")

print(f"Original rows: {len(df)}")
print(f"After deduplication: {len(df_deduped)}")
print(f"Removed: {len(df) - len(df_deduped)} duplicate rows")

# Save deduplicated version
df_deduped.to_parquet(DATA_FOLDER / "metadata_deduped.parquet")

         image_filename                                              latex  \
2664061  10220a1920.png  { \cal H } _ { \Delta , l } = \frac { 1 } { x ...   
3319167  10220a1920.png  { \cal H } _ { \Delta , l } = \frac { 1 } { x ...   

         split image_type                                              typst  
2664061  train    printed  cal( H ) _ ( Delta , l ) = ( 1 ) / ( x _ ( 12 ...  
3319167   test    printed  cal( H ) _ ( Delta , l ) = ( 1 ) / ( x _ ( 12 ...  

        image_filename                                              latex  \
156620   105_edwin.bmp  \frac { b ^ { 2 } c ^ { 2 } - 4 b ^ { 3 } d - ...   
3324611  105_edwin.bmp  \frac { b ^ { 2 } c ^ { 2 } - 4 b ^ { 3 } d - ...   

              split   image_type  \
156620        train  handwritten   
3324611  validation  handwritten   

                                                     typst  
156620   ( b ^ ( 2 ) c ^ ( 2 ) - 4 b ^ ( 3 ) d - 4 a c ...  
3324611  ( b ^ ( 2 ) c ^ ( 2 ) - 4 b ^ ( 3 ) d - 4 a c ...  

  